# Pandas

- library for Data Analysis and Manipulation

**Why Pandas?**

- provides ability to work with Tabular data
  - `Tabular Data` : data that is organized into tables having rows and cols

In [1]:
import pandas as pd
import numpy as np
# jupyter nbconvert --to markdown pandas.ipynb --output README.md

## `Series` objects

- A `Series` object is 1D array that can hold/store data.

### Creating a `Series`

In [9]:
l = ["C", "C++", "Python", "Javascript"]
s = pd.Series(l)
s

0             C
1           C++
2        Python
3    Javascript
dtype: object

### Similar to a 1D `ndarray`

`Series` objects behave much like one-dimensional NumPy `ndarray`s, and you can often pass them as parameters to NumPy functions:

In [3]:
import numpy as np

s = pd.Series([2,4,6,8])
np.exp(s)

0       7.389056
1      54.598150
2     403.428793
3    2980.957987
dtype: float64

Arithmetic operations on `Series` are also possible, and they apply *elementwise*, just like for `ndarray`s:

In [4]:
s + [1000,2000,3000,4000]

0    1002
1    2004
2    3006
3    4008
dtype: int64

Similar to NumPy, if you add a single number to a `Series`, that number is added to all items in the `Series`. This is called * broadcasting*:

In [5]:
s + 1000

0    1002
1    1004
2    1006
3    1008
dtype: int64

The same is true for all binary operations such as `*` or `/`, and even conditional operations:

In [6]:
s < 0

0    False
1    False
2    False
3    False
dtype: bool

### Indexing

In [28]:
s2 = pd.Series(l, index=["a", "b", "c", "d"])
s2

a             C
b           C++
c        Python
d    Javascript
dtype: object

You can then use the `Series` just like a `dict`:

In [11]:
s2["b"]

'C++'

You can still access the items by integer location, like in a regular array:

In [12]:
s2[1]

'C++'

To make it clear when you are accessing by label or by integer location, it is recommended to always use the `loc` attribute when accessing by label, and the `iloc` attribute when accessing by integer location:

In [13]:
s2.loc["b"]

'C++'

In [14]:
s2.iloc[1]

'C++'

#### Slicing a `Series` also slices the index labels:

In [15]:
s2.iloc[1:3]

b       C++
c    Python
dtype: object

This can lead to unexpected results when using the default numeric labels, so be careful:

In [16]:
surprise = pd.Series([1000, 1001, 1002, 1003])
surprise

0    1000
1    1001
2    1002
3    1003
dtype: int64

In [17]:
surprise_slice = surprise[2:]
surprise_slice

2    1002
3    1003
dtype: int64

Oh look! The first element has index label `2`. The element with index label `0` is absent from the slice:

In [18]:
try:
    surprise_slice[0]
except KeyError as e:
    print("Key error:", e)

Key error: 0


But remember that you can access elements by integer location using the `iloc` attribute. This illustrates another reason why it's always better to use `loc` and `iloc` to access `Series` objects:

In [19]:
surprise_slice.iloc[0]

1002

#### Init from `dict`

You can create a `Series` object from a `dict`. The keys will be used as index labels:

In [20]:
weights = {"a": 68, "b": 83, "c": 86, "d": 68}
s3 = pd.Series(weights)
s3

a    68
b    83
c    86
d    68
dtype: int64

You can control which elements you want to include in the `Series` and in what order by explicitly specifying the desired `index`:

In [24]:
s4 = pd.Series(weights, index = ["c", "a"])
s4

c    86
a    68
dtype: int64

### Automatic alignment

When an operation involves multiple `Series` objects, `pandas` automatically aligns items by matching index labels.

In [34]:
s2 = pd.Series([1,2,3], index=["a", "b", "c"])
s3 = pd.Series([10,20,40], index=["a", "b", "d"])

print(s2.keys())
print(s3.keys())

s2 + s3


Index(['a', 'b', 'c'], dtype='object')
Index(['a', 'b', 'd'], dtype='object')


a    11.0
b    22.0
c     NaN
d     NaN
dtype: float64

The resulting `Series` contains the union of index labels from `s2` and `s3`. Since `"d"` is missing from `s2` and `"c"` is missing from `s3`, these items have a `NaN` result value. (ie. Not-a-Number means *missing*).

Automatic alignment is very handy when working with data that may come from various sources with varying structure and missing items. But if you forget to set the **right index labels**, you can have surprising results:

In [36]:
s5 = pd.Series([1000,1000,1000,1000])
s2 + s5

a   NaN
b   NaN
c   NaN
0   NaN
1   NaN
2   NaN
3   NaN
dtype: float64

Pandas could not align the `Series`, since their labels do not match at all, hence the full `NaN` result.

### Init with a scalar

You can also initialize a `Series` object using a scalar and a list of index labels: all items will be set to the scalar.

In [37]:
meaning = pd.Series(42, ["a", "b", "c"])
meaning

a    42
b    42
c    42
dtype: int64

In [38]:
s6 = pd.Series([83, 68], index=["a", "b"], name="weights")
s6

a    83
b    68
Name: weights, dtype: int64

## `DataFrame` objects

- A DataFrame object represents a 2d labelled array, with cell values, column names and row index labels
- You can see `DataFrame`s as dictionaries of `Series`.


### Creating a `DataFrame`

#### from numpy array

In [49]:
import numpy as np

In [51]:
arr = np.random.randint(10,100,size=(6,4))
arr

array([[30, 27, 82, 14],
       [94, 66, 75, 56],
       [53, 19, 72, 20],
       [32, 91, 10, 14],
       [88, 65, 70, 49],
       [31, 57, 27, 95]])

In [53]:
df = pd.DataFrame(data=arr)
df

,0,1,2,3
0,30,27,82,14
1,94,66,75,56
2,53,19,72,20
3,32,91,10,14
4,88,65,70,49
5,31,57,27,95


In [55]:
type(df)

pandas.core.frame.DataFrame

In [57]:
df[2]

0    82
1    75
2    72
3    10
4    70
5    27
Name: 2, dtype: int32

In [58]:
type(df[2])

pandas.core.series.Series

In [59]:
type(df[0])

pandas.core.series.Series

#### dictionary of `pd.Series` or `List` objects:

In [49]:
data = {
	'roll_no': [10,3,2,4],
	'sid':[111,112,113,114],
	'marks':[90,80,70,60]
}
df = pd.DataFrame(data)
df

,roll_no,sid,marks
0,10,111,90
1,3,112,80
2,2,113,70
3,4,114,60


In [28]:
people_dict = {
    "weight": pd.Series([68, 83, 112], index=["alice", "bob", "charles"]),
    "birthyear": pd.Series([1984, 1985, 1992], index=["bob", "alice", "charles"], name="year"),
    "children": pd.Series([0, 3], index=["charles", "bob"]),
    "hobby": pd.Series(["Biking", "Dancing"], index=["alice", "bob"]),
}
people = pd.DataFrame(people_dict)
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


A few things to note:
* the `Series` were automatically aligned based on their index,
* missing values are represented as `NaN`,
* `Series` names are ignored (the name `"year"` was dropped),
* `DataFrame`s are displayed nicely in Jupyter notebooks, woohoo!

You can access columns pretty much as you would expect. They are returned as `Series` objects:

In [51]:
people["birthyear"]

alice      1985
bob        1984
charles    1992
Name: birthyear, dtype: int64

You can also get multiple columns at once:

In [52]:
people[["birthyear", "hobby"]]

,birthyear,hobby
alice,1985,Biking
bob,1984,Dancing
charles,1992,NaN


It is also possible to create a `DataFrame` with a dictionary (or list) of dictionaries (or list):

In [61]:
people = pd.DataFrame({
    "birthyear": {"alice": 1985, "bob": 1984, "charles": 1992},
    "hobby": {"alice": "Biking", "bob": "Dancing"},
    "weight": {"alice": 68, "bob": 83, "charles": 112},
    "children": {"bob": 3, "charles": 0}
})
people


,birthyear,hobby,weight,children
alice,1985,Biking,68,NaN
bob,1984,Dancing,83,3.0
charles,1992,NaN,112,0.0


#### `DataFrame(columns=[],index=[])` constructor

If you pass a list of columns and/or index row labels to the `DataFrame` constructor, it will guarantee that these columns and/or rows will exist, in that order, and no other column/row will exist. For example:

In [6]:
d2 = pd.DataFrame(
        people_dict,
        columns=["birthyear", "weight", "height"],
        index=["bob", "alice", "eugene"]
     )
d2

,birthyear,weight,height
bob,1984.0,83.0,NaN
alice,1985.0,68.0,NaN
eugene,NaN,NaN,NaN


Another convenient way to create a `DataFrame` is to pass all the values to the constructor as an `ndarray`, or a list of lists, and specify the column names and row index labels separately:

In [10]:
values = [
            [1985, np.nan, "Biking",   68],
            [1984, 3,      "Dancing",  83],
            [1992, 0,      np.nan,    112]
         ]
d3 = pd.DataFrame(
        values,
        columns=["birthyear", "children", "hobby", "weight"],
        index=["alice", "bob", "charles"]
     )
d3

,birthyear,children,hobby,weight
alice,1985,NaN,Biking,68
bob,1984,3.0,Dancing,83
charles,1992,0.0,NaN,112


To specify missing values, you can either use `np.nan` or NumPy's masked arrays:

In [55]:
masked_array = np.ma.asarray(values, dtype=np.object)
masked_array[(0, 2), (1, 2)] = np.ma.masked
d3 = pd.DataFrame(
        masked_array,
        columns=["birthyear", "children", "hobby", "weight"],
        index=["alice", "bob", "charles"]
     )
d3

,birthyear,children,hobby,weight
alice,1985,NaN,Biking,68
bob,1984,3,Dancing,83
charles,1992,0,NaN,112


Instead of an `ndarray`, you can also pass a `DataFrame` object:

In [11]:
d4 = pd.DataFrame(
         d3,
         columns=["hobby", "children"],
         index=["alice", "bob"]
     )
d4

,hobby,children
alice,Biking,NaN
bob,Dancing,3.0


### Indexing, Masking, Query

#### Extracting Columns

In [2]:
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d"])
# df.columns = ["a", "b", "c", "d"]
df

,a,b,c,d
0,31,86,98,71
1,31,13,43,20
2,22,24,81,85
3,57,77,83,99
4,22,13,97,49
5,91,87,36,88


In [3]:
df['c']

0    98
1    43
2    81
3    83
4    97
5    36
Name: c, dtype: int32

In [5]:
df.c

0    98
1    43
2    81
3    83
4    97
5    36
Name: c, dtype: int32

multiple columns can be extracted at once:

In [6]:
df[['b','c','a']]

,b,c,a
0,86,98,31
1,13,43,31
2,24,81,22
3,77,83,57
4,13,97,22
5,87,36,91


#### Extracting Rows

In [50]:
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr)
df.columns = ["a", "b", "c", "d"]
df.index = "p q r s t u".split()
df

,a,b,c,d
p,88,61,34,34
q,11,68,61,92
r,91,54,20,58
s,63,63,82,41
t,82,41,76,98
u,73,52,58,10


##### `loc()` - label location

The `loc` attribute lets you access rows instead of columns. The result is a `Series` object in which the `DataFrame`'s column names are mapped to row index labels:

In [51]:
df.loc["p"]

a    88
b    61
c    34
d    34
Name: p, dtype: int32

##### `iloc()` - integer location

You can also access rows by integer location using the `iloc` attribute:

In [52]:
df.iloc[2]

a    91
b    54
c    20
d    58
Name: r, dtype: int32

#### Extracting Rows with Columns using slice

In [53]:
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr)
df.columns = ["a", "b", "c", "d"]
df.index = "p q r s t u".split()
df


,a,b,c,d
p,60,53,17,23
q,62,68,79,49
r,52,99,19,47
s,13,51,64,31
t,98,79,80,21
u,61,41,81,25


In [57]:
# df.iloc[1:3]
df.iloc[1:3,:]

,a,b,c,d
q,62,68,79,49
r,52,99,19,47


In [60]:
df.iloc[1:3,0:2]

,a,b
q,62,68
r,52,99


In [58]:
df.iloc[1:3][['a','b']]

,a,b
q,62,68
r,52,99


In [150]:
df.iloc[1:3,:2]

,a,b
q,17,52
r,81,76


#### Masking - Boolean Indexing

In [156]:
df

,a,b,c,d
p,64,81,95,79
q,17,52,83,41
r,81,76,54,29
s,96,58,22,98
t,83,27,67,95
u,16,34,60,11


In [157]:
df > 30

,a,b,c,d
p,True,True,True,True
q,False,True,True,True
r,True,True,True,False
s,True,True,False,True
t,True,False,True,True
u,False,True,True,False


In [158]:
mask = df > 30
df[mask]

,a,b,c,d
p,64.0,81.0,95.0,79.0
q,NaN,52.0,83.0,41.0
r,81.0,76.0,54.0,NaN
s,96.0,58.0,NaN,98.0
t,83.0,NaN,67.0,95.0
u,NaN,34.0,60.0,NaN


This is most useful when combined with boolean expressions:

In [159]:
df['a'] <50 # d.a < 50

p    False
q     True
r    False
s    False
t    False
u     True
Name: a, dtype: bool

In [154]:
df[df["a"] < 50] # only getting q and u as both is True

,a,b,c,d
q,17,52,83,41
u,16,34,60,11


In [160]:
df[df["a"] < 50][['a','d']]

,a,d
q,17,41
u,16,11


#### Querying a `DataFrame`

The `query()` method lets you filter a `DataFrame` based on a query expression:

In [87]:
people.query("age > 30 and pets == 0")

,hobby,height,weight,age,over 30,pets,body_mass_index,overweight
bob,Dancing,181,83,34,True,0.0,25.335002,False


### Adding and Removing Row/Columns

####  Adding Column

In [71]:
people_dict = {
    "weight": pd.Series([68, 83, 112], index=["alice", "bob", "charles"]),
    "birthyear": pd.Series([1984, 1985, 1992], index=["bob", "alice", "charles"], name="year"),
    "children": pd.Series([0, 3], index=["charles", "bob"]),
    "hobby": pd.Series(["Biking", "Dancing"], index=["alice", "bob"]),
}
people = pd.DataFrame(people_dict)
people


,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


##### direct assignment

In [72]:
people["age"] = 2021 - people["birthyear"]  # adds a new column "age"
people["over 30"] = people["age"] > 30      # adds another column "over 30"
people

,weight,birthyear,children,hobby,age,over 30
alice,68,1985,NaN,Biking,36,True
bob,83,1984,3.0,Dancing,37,True
charles,112,1992,0.0,NaN,29,False


In [73]:
birthyears

alice      1985
bob        1984
charles    1992
Name: birthyear, dtype: int64

When you add a new colum, it must have the same number of rows. Missing rows are filled with NaN, and extra rows are ignored:

In [74]:
people["pets"] = pd.Series({"bob": 0, "charles": 5, "eugene":1})  # alice is missing, eugene is ignored
people

,weight,birthyear,children,hobby,age,over 30,pets
alice,68,1985,NaN,Biking,36,True,NaN
bob,83,1984,3.0,Dancing,37,True,0.0
charles,112,1992,0.0,NaN,29,False,5.0


##### `insert(position,column,value)`

When adding a new column, it is added at the end (on the right) by default. You can also insert a column anywhere else using the `insert()` method:

In [75]:
people.insert(1, "height", [172, 181, 185])
people

,weight,height,birthyear,children,hobby,age,over 30,pets
alice,68,172,1985,NaN,Biking,36,True,NaN
bob,83,181,1984,3.0,Dancing,37,True,0.0
charles,112,185,1992,0.0,NaN,29,False,5.0


##### `assign()`: Assigning new columns

You can also create new columns by calling the `assign()` method. Note that this returns a new `DataFrame` object, **the original is not modified:**

In [76]:
people.assign(
    body_mass_index = people["weight"] / (people["height"] / 100) ** 2,
    has_pets = people["pets"] > 0
)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,has_pets
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,False
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


In [77]:
people # the original is not modified

,weight,height,birthyear,children,hobby,age,over 30,pets
alice,68,172,1985,NaN,Biking,36,True,NaN
bob,83,181,1984,3.0,Dancing,37,True,0.0
charles,112,185,1992,0.0,NaN,29,False,5.0


Note that you cannot access columns created within the same assignment:

In [78]:
try:
    people.assign(
        body_mass_index = people["weight"] / (people["height"] / 100) ** 2,
        overweight = people["body_mass_index"] > 25 # body_mass_index is not defined at this point
    )
except KeyError as e:
    print("Key error:", e)

Key error: 'body_mass_index'


The solution is to split this assignment in two consecutive assignments:

In [79]:
d6 = people.assign(body_mass_index = people["weight"] / (people["height"] / 100) ** 2)
d6.assign(overweight = d6["body_mass_index"] > 25)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,overweight
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,True
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


Having to create a temporary variable `d6` is not very convenient. You may want to just chain the assigment calls, but it does not work because the `people` object is not actually modified by the first assignment:

In [80]:
try:
    (people
         .assign(body_mass_index = people["weight"] / (people["height"] / 100) ** 2)
         .assign(overweight = people["body_mass_index"] > 25)
    )
except KeyError as e:
    print("Key error:", e)

Key error: 'body_mass_index'


But fear not, there is a simple solution. You can pass a function to the `assign()` method (typically a `lambda` function), and this function will be called with the `DataFrame` as a parameter:

In [81]:
(people
     .assign(body_mass_index = lambda df: df["weight"] / (df["height"] / 100) ** 2)
     .assign(overweight = lambda df: df["body_mass_index"] > 25)
)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,overweight
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,True
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


Problem solved!

#### Adding Row

In [39]:
arr= np.random.randint(10, 100, size=(6,4))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d"])
df.index = "p q r s t u".split()

In [42]:
print(df.loc['p'])
print(df.iloc[0])

a    81
b    86
c    65
d    23
Name: p, dtype: int32
a    81
b    86
c    65
d    23
Name: p, dtype: int32


In [44]:
df.loc['x'] = [1,2,3,4]
df.tail()

,a,b,c,d
r,79,12,25,51
s,10,99,54,73
t,98,55,14,90
u,61,62,63,55
x,1,2,3,4


#### Removing Rows/Columns

- `drop` method:
  - `drop(columns,axis=1)`
  - `drop(index,axis=0)`
  - `drop(labels,axis=1)`
  - `drop(labels,axis=0)`
-  `pop`

In [18]:
arr = np.random.randint(10, 100, size=(4,8))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d", "e", "f", "g", "h"])
df['a+b'] = df['a'] + df['b']
df['a-b'] = df['a'] * df['b']
df

,a,b,c,d,e,f,g,h,a+b,a-b
0,53,98,47,62,69,88,48,22,151,5194
1,58,11,46,71,46,26,61,49,69,638
2,85,97,64,76,38,56,67,94,182,8245
3,53,51,29,23,60,36,46,28,104,2703


In [19]:
delC = df.pop('c')  # removes column c
del df["d"] # removes column d
df

,a,b,e,f,g,h,a+b,a-b
0,53,98,69,88,48,22,151,5194
1,58,11,46,26,61,49,69,638
2,85,97,38,56,67,94,182,8245
3,53,51,60,36,46,28,104,2703


In [20]:
df.drop(columns=['e','f','a-b']) # removes columns e, f, a-b

,a,b,g,h,a+b
0,53,98,48,22,151
1,58,11,61,49,69
2,85,97,67,94,182
3,53,51,46,28,104


In [21]:
df

,a,b,e,f,g,h,a+b,a-b
0,53,98,69,88,48,22,151,5194
1,58,11,46,26,61,49,69,638
2,85,97,38,56,67,94,182,8245
3,53,51,60,36,46,28,104,2703


In [22]:
df.drop(columns=['e', 'f', 'a-b'], inplace=True)  # original df is modified

In [23]:
df

,a,b,g,h,a+b
0,53,98,48,22,151
1,58,11,61,49,69
2,85,97,67,94,182
3,53,51,46,28,104


In [24]:
df.drop(1,inplace=True) # removes row 1
df

,a,b,g,h,a+b
0,53,98,48,22,151
2,85,97,67,94,182
3,53,51,46,28,104


In [25]:
df.index

Int64Index([0, 2, 3], dtype='int64')

In [28]:
df.drop(df.index[[0,2]])


,a,b,g,h,a+b
2,85,97,67,94,182


### Handy Methods and Properties

In [30]:
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr)
df.columns = 'a b c d'.split()
df

,a,b,c,d
0,20,34,45,36
1,22,37,11,65
2,36,17,68,48
3,77,69,99,46
4,83,28,34,67
5,46,24,28,11


#### `shape` , `dtypes` , `info()`, `describe()`

In [31]:
df.shape

(6, 4)

In [6]:
df.dtypes

a    int32
b    int32
c    int32
d    int32
dtype: object

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       6 non-null      int32
 1   b       6 non-null      int32
 2   c       6 non-null      int32
 3   d       6 non-null      int32
dtypes: int32(4)
memory usage: 224.0 bytes


In [33]:
df.describe()

,a,b,c,d
count,6.000000,6.000000,6.000000,6.000000
mean,47.333333,34.833333,47.500000,45.500000
std,27.097355,18.192489,31.538865,20.637345
min,20.000000,17.000000,11.000000,11.000000
25%,25.500000,25.000000,29.500000,38.500000
50%,41.000000,31.000000,39.500000,47.000000
75%,69.250000,36.250000,62.250000,60.750000
max,83.000000,69.000000,99.000000,67.000000


In [34]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
a,6.0,47.333333,27.097355,20.0,25.5,41.0,69.25,83.0
b,6.0,34.833333,18.192489,17.0,25.0,31.0,36.25,69.0
c,6.0,47.500000,31.538865,11.0,29.5,39.5,62.25,99.0
d,6.0,45.500000,20.637345,11.0,38.5,47.0,60.75,67.0


In [36]:
df['a'].describe()

count     6.000000
mean     47.333333
std      27.097355
min      20.000000
25%      25.500000
50%      41.000000
75%      69.250000
max      83.000000
Name: a, dtype: float64

#### `head` and `tail`

- `head`: prints the first 5 rows
- `tail`: prints the last 5 rows

In [44]:
df.head()

,0,1,2,3
0,80,11,76,81
1,51,29,24,59
2,64,64,26,62
3,38,29,78,97
4,10,72,77,45


In [45]:
df.head(n=3)

,0,1,2,3
0,80,11,76,81
1,51,29,24,59
2,64,64,26,62


In [47]:
df.tail(n=2)

,0,1,2,3
4,10,72,77,45
5,83,55,26,71


#### `columns`

In [5]:
df.columns

Index(['a', 'b', 'c', 'd'], dtype='object')

#### `values` : returns a numpy array

In [162]:
arr = df.values
arr

array([[96, 71, 41, 34],
       [43, 92, 89, 79],
       [20, 28, 78, 42],
       [51, 82, 67, 30],
       [13, 27, 72, 79],
       [85, 12, 62, 14]])

#### `unique` and `nunique`

The Pandas Unique technique identifies the unique values of a Pandas Series.

In [11]:
people_dict = {
    "country": pd.Series(['BD','IN','PAK','BD','BD','IN']),
	"name":pd.Series(['A','B','C','D','E','F'])
}
people = pd.DataFrame(people_dict)
people

,country,name
0,BD,A
1,IN,B
2,PAK,C
3,BD,D
4,BD,E
5,IN,F


In [16]:
people.nunique()

country    3
name       6
dtype: int64

In [20]:
people['country'].nunique()


3

In [19]:
people['country'].unique()


array(['BD', 'IN', 'PAK'], dtype=object)

#### `value_counts()`

count occupance of each unique element

In [22]:
people['country'].value_counts()

BD     3
IN     2
PAK    1
Name: country, dtype: int64

In [23]:
people['country'].value_counts()['BD']

3

#### Sorting a `DataFrame`
You can sort a `DataFrame` by calling its `sort_index` method. By default it sorts the rows by their **index label**, in ascending order, but let's reverse the order:

In [37]:
people_dict = {
    "country": pd.Series(['BD', 'IN', 'PAK', 'SL', 'US', 'IN']),
   	"name": pd.Series(['A', 'B', 'C', 'D', 'E', 'F']),
	   "cgpa":pd.Series([3.56, 4.00, 3.55, 3.86, 3.99, 3.89])
}
people = pd.DataFrame(people_dict)
people

,country,name,cgpa
0,BD,A,3.56
1,IN,B,4.00
2,PAK,C,3.55
3,SL,D,3.86
4,US,E,3.99
5,IN,F,3.89


In [38]:
people.sort_index(ascending=False).head(n=3)

,country,name,cgpa
5,IN,F,3.89
4,US,E,3.99
3,SL,D,3.86


Note that `sort_index` returned a sorted *copy* of the `DataFrame`. To modify `people` directly, we can set the `inplace` argument to `True`. Also, we can sort the columns instead of the rows by setting `axis=1`:

In [39]:
people.sort_index(axis=1,ascending=False, inplace=True)
people

,name,country,cgpa
0,A,BD,3.56
1,B,IN,4.00
2,C,PAK,3.55
3,D,SL,3.86
4,E,US,3.99
5,F,IN,3.89


To sort the `DataFrame` by the values instead of the labels, we can use `sort_values` and specify the column to sort by:

In [41]:
people.sort_values(by=["name"], ascending=False,inplace=True)
people

,name,country,cgpa
5,F,IN,3.89
4,E,US,3.99
3,D,SL,3.86
2,C,PAK,3.55
1,B,IN,4.00
0,A,BD,3.56


In [46]:
people.sort_values(by=["cgpa", "name"])

,name,country,cgpa
2,C,PAK,3.55
0,A,BD,3.56
3,D,SL,3.86
5,F,IN,3.89
4,E,US,3.99
1,B,IN,4.00


#### `apply`

Pandas.apply allow the users to pass a function and apply it on every single value of the Pandas series. It comes as a huge improvement for the pandas library as this function helps to segregate data according to the conditions required due to which it is efficiently used in data science and machine learning.

In [47]:
df

,a,b,c,d
0,74,75,90,63
1,95,57,33,56
2,23,27,14,24
3,64,88,45,81
4,72,17,59,65
5,87,83,98,91


In [48]:
df.apply(lambda x: x+x)

,a,b,c,d
0,148,150,180,126
1,190,114,66,112
2,46,54,28,48
3,128,176,90,162
4,144,34,118,130
5,174,166,196,182


In [50]:
df['a'].apply(lambda x:x*10)

0    740
1    950
2    230
3    640
4    720
5    870
Name: a, dtype: int64

More in [Handling String Data - Converting Reality to Numbers](#handling-string-data---converting-reality-to-numbers)


### Saving & loading files

Pandas can save `DataFrame`s to various backends, including file formats such as CSV, Excel, JSON, HTML and HDF5, or to a SQL database. Let's create a `DataFrame` to demonstrate this:

In [57]:
my_df = pd.DataFrame(
    [["Biking", 68.5, 1985, np.nan], ["Dancing", 83.1, 1984, 3]], 
    columns=["hobby","weight","birthyear","children"],
    index=["alice", "bob"]
)
my_df

,hobby,weight,birthyear,children
alice,Biking,68.5,1985,NaN
bob,Dancing,83.1,1984,3.0


#### Saving
Let's save it to CSV, HTML and JSON:

In [58]:
my_df.to_csv("my_df.csv")
# my_df.to_csv("my_df.csv",index=False)
my_df.to_html("my_df.html")
my_df.to_json("my_df.json")

#### Loading
Now let's load our CSV file back into a `DataFrame`:

In [62]:
my_df_loaded = pd.read_csv("my_df.csv")
my_df_loaded

,Unnamed: 0,hobby,weight,birthyear,children
0,alice,Biking,68.5,1985,NaN
1,bob,Dancing,83.1,1984,3.0


In [63]:
my_df_loaded = pd.read_csv("my_df.csv",index_col=0)
my_df_loaded


,hobby,weight,birthyear,children
alice,Biking,68.5,1985,NaN
bob,Dancing,83.1,1984,3.0


As you might guess, there are similar `read_json`, `read_html`, `read_excel` functions as well.  We can also read data straight from the Internet. For example, let's load the top 1,000 U.S. cities from github:

In [64]:
us_cities = None
try:
    csv_url = "https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv"
    us_cities = pd.read_csv(csv_url, index_col=0)
    us_cities = us_cities.head()
except IOError as e:
    print(e)
us_cities

,State,Population,lat,lon
City,,,,
Marysville,Washington,63269,48.051764,-122.177082
Perris,California,72326,33.782519,-117.228648
Cleveland,Ohio,390113,41.499320,-81.694361
Worcester,Massachusetts,182544,42.262593,-71.802293
Columbia,South Carolina,133358,34.000710,-81.034814


### Operations on `DataFrame`s

Although `DataFrame`s do not try to mimick NumPy arrays, there are a few similarities. Let's create a `DataFrame` to demonstrate this:

In [101]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
df


,roll_no,ppr_id,marks
0,83,1930,83
1,45,1954,57
2,92,1638,67
3,92,1126,91
4,56,1369,96


In [114]:
df['marks'].sum()

394

In [115]:
df['marks'].mean()

78.8

In [116]:
df['marks'].cumsum()

0     83
1    140
2    207
3    298
4    394
Name: marks, dtype: int32

In [117]:
df['roll_no'].count()

5

In [118]:
df['marks'].min()

57

In [119]:
df['marks'].max()

96

In [120]:
df['marks'].var()

269.2

In [121]:
df['marks'].std()

16.407315441594946

In [122]:
df.corr()

,roll_no,ppr_id,marks
roll_no,1.000000,-0.352082,0.25746
ppr_id,-0.352082,1.000000,-0.70311
marks,0.257460,-0.703110,1.00000


Similarly, adding a single value to a `DataFrame` will add that value to all elements in the `DataFrame`. This is called *broadcasting*:

In [102]:
df + 1

,roll_no,ppr_id,marks
0,84,1931,84
1,46,1955,58
2,93,1639,68
3,93,1127,92
4,57,1370,97


Of course, the same is true for all other binary operations, including arithmetic (`*`,`/`,`**`...) and conditional (`>`, `==`...) operations:

In [104]:
df >= 500

,roll_no,ppr_id,marks
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False


Aggregation operations, such as computing the `max`, the `sum` or the `mean` of a `DataFrame`, apply to each column, and you get back a `Series` object:

In [105]:
df.mean()

roll_no      73.6
ppr_id     1603.4
marks        78.8
dtype: float64

The `all` method is also an aggregation operation: it checks whether all values are `True` or not. Let's see during which months all students got a grade greater than `5`:

In [106]:
(df > 50).all()

roll_no    False
ppr_id      True
marks       True
dtype: bool

Most of these functions take an optional `axis` parameter which lets you specify along which axis of the `DataFrame` you want the operation executed. The default is `axis=0`, meaning that the operation is executed vertically (on each column). You can set `axis=1` to execute the operation horizontally (on each row). For example, let's find out which students had all grades greater than `5`:

In [108]:
(df > 50).all(axis = 1)

0     True
1    False
2     True
3     True
4     True
dtype: bool

The `any` method returns `True` if any value is True. Let's see who got at least one grade 10:

In [110]:
(df == 92).any(axis = 1)

0    False
1    False
2     True
3     True
4    False
dtype: bool

If you add a `Series` object to a `DataFrame` (or execute any other binary operation), pandas attempts to broadcast the operation to all *rows* in the `DataFrame`. This only works if the `Series` has the same size as the `DataFrame`s rows. For example, let's subtract the `mean` of the `DataFrame` (a `Series` object) from the `DataFrame`:

In [111]:
df - df.mean()

,roll_no,ppr_id,marks
0,9.4,326.6,4.2
1,-28.6,350.6,-21.8
2,18.4,34.6,-11.8
3,18.4,-477.4,12.2
4,-17.6,-234.4,17.2


If you want to subtract the global mean from every grade, here is one way to do it:

In [112]:
df - df.values.mean() # subtracts the global mean 

,roll_no,ppr_id,marks
0,-502.266667,1344.733333,-502.266667
1,-540.266667,1368.733333,-528.266667
2,-493.266667,1052.733333,-518.266667
3,-493.266667,540.733333,-494.266667
4,-529.266667,783.733333,-489.266667


### Aggregating with `groupby`

Similar to the SQL language, pandas allows grouping your data into groups to run calculations over each group.

First, let's add some extra data about each person so we can group them, and let's go back to the `final_grades` `DataFrame` so we can see how `NaN` values are handled:

In [67]:
iris = pd.read_csv("iris.csv")
iris.head()


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [68]:
iris.aggregate('min')

sepal_length       4.3
sepal_width        2.0
petal_length       1.0
petal_width        0.1
species         setosa
dtype: object

In [69]:
iris.aggregate(['min','max','mean','median'])

,sepal_length,sepal_width,petal_length,petal_width,species
min,4.300000,2.000,1.000000,0.100000,setosa
max,7.900000,4.400,6.900000,2.500000,virginica
mean,5.843333,3.054,3.758667,1.198667,NaN
median,5.800000,3.000,4.350000,1.300000,NaN


In [70]:
groupby = iris.groupby('species')
groupby

In [71]:
groupby.min()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,4.3,2.3,1.0,0.1
versicolor,4.9,2.0,3.0,1.0
virginica,4.9,2.2,4.5,1.4


In [72]:
groupby.mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.418,1.464,0.244
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [77]:
iris[iris['species'] == 'setosa']['sepal_length'].mean()

5.005999999999999

### Handling Missing Data

- `dropna`
- `fillna`

In [71]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id':np.random.randint(1000, 2000, size=5),
	'marks':np.random.randint(50,100,size=5)
}
df = pd.DataFrame(data)
df

,roll_no,ppr_id,marks
0,37,1690,56
1,15,1700,87
2,53,1364,55
3,94,1372,90
4,44,1291,76


In [72]:
nan_idx = np.random.randint(0,5,3)
df['marks'][nan_idx] = np.nan
# df['marks'][[1,4,3]] = np.nan
df

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,NaN
2,53,1364,55.0
3,94,1372,NaN
4,44,1291,NaN


#### `isnull()` and `notnull()`

In [73]:
df.isnull()

,roll_no,ppr_id,marks
0,False,False,False
1,False,False,True
2,False,False,False
3,False,False,True
4,False,False,True


In [75]:
df.isnull().sum()

roll_no    0
ppr_id     0
marks      3
dtype: int64

In [74]:
df['marks'].isnull()

0    False
1     True
2    False
3     True
4     True
Name: marks, dtype: bool

#### `fillna`

In [77]:
df.fillna("FILLED")

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,FILLED
2,53,1364,55.0
3,94,1372,FILLED
4,44,1291,FILLED


In [78]:
df

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,NaN
2,53,1364,55.0
3,94,1372,NaN
4,44,1291,NaN


In [80]:
df = df.fillna("FILLED")
df.fillna("FILLED",inplace=True)
df


,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,FILLED
2,53,1364,55.0
3,94,1372,FILLED
4,44,1291,FILLED


In [82]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
nan_idx = np.random.randint(0, 5, 3)
df['marks'][nan_idx] = np.nan
df

,roll_no,ppr_id,marks
0,60,1764,53.0
1,58,1443,NaN
2,23,1603,96.0
3,18,1626,NaN
4,21,1656,NaN


In [84]:
df.marks.fillna(df.marks.mean(), inplace=True)

In [85]:
df

,roll_no,ppr_id,marks
0,60,1764,53.0
1,58,1443,74.5
2,23,1603,96.0
3,18,1626,74.5
4,21,1656,74.5


#### `dropna`

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
df['marks'][[0, 2, 4]] = np.nan
df['roll_no'][[0, 2]] = np.nan
df


,roll_no,ppr_id,marks
0,NaN,1074,NaN
1,4.0,1867,60.0
2,NaN,1103,NaN
3,90.0,1699,54.0
4,70.0,1372,NaN


In [99]:
df.dropna()

,roll_no,ppr_id,marks
1,4.0,1867,60.0
3,90.0,1699,54.0


### Handling String Data - Converting Reality to Numbers

In [45]:
people_dict = {
	# panda series from a list of A to D
	"name":pd.Series(['A','B','C','D']),
	"gender":pd.Series(['Female','Male','Female','Female'])
}
people = pd.DataFrame(people_dict)
people

,name,gender
0,A,Female
1,B,Male
2,C,Female
3,D,Female


In [47]:
def f(g):
	if g == 'Male':
		return 0
	else:
		return 1


people['sex'] = people.gender.apply(f)
people

,name,gender,sex
0,A,Female,1
1,B,Male,0
2,C,Female,1
3,D,Female,1
